In [24]:
import pandas as pd
import wandb
from tqdm.auto import tqdm

/Users/mag7273/mambaforge/envs/etnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
api = wandb.Api()

entity = "ten-harvard"
project = "spatial-cc"
runs = api.runs(entity + "/" + project)

In [26]:
rows = []
histories = []
for run in tqdm(runs):
    row = {}
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    row.update(**run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    row["config"] = {k: v for k, v in run.config.items() if not k.startswith("_")}

    # .name is the human-readable name of the run.
    row["name"] = run.name

    # extract seed
    row["seed"] = row['config'].get("seed", None)

    # extract arch
    row["arch"] = row["config"].get("arch_name", None)

    # baseline name
    row["baseline"] = row["config"].get("baseline_name", None)

    # ckpt_prefix
    row["ckpt_prefix"] = row["config"].get("ckpt_prefix", None)

    # history
    row["history"] = run.history()

    rows.append(row)

runs_df = pd.DataFrame(rows)
runs_df

100%|██████████| 903/903 [05:04<00:00,  2.96it/s]


,best_model_test_mse,lr,_step,train_loss,best_val_r2,best_model_test_r2,val_r2,test_r2,_runtime,_timestamp,...,name,seed,arch,baseline,ckpt_prefix,history,_wandb,scatter,best_r2,eval_loss
0,0.809440,0.009994,8.0,0.324672,0.014962,0.033230,0.003756,0.051446,718.011611,1.716394e+09,...,cc2-fix_equivariant-virtual_378_w69pf0ua,378.0,arch-cc2-1,equivariant-virtual,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
1,0.802445,0.009994,8.0,0.327559,0.018571,0.045479,0.007354,0.033446,745.481871,1.716394e+09,...,cc2-fix_equivariant-ho_378_wl3jazrl,378.0,arch-cc2-1,equivariant-ho,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
2,0.766299,0.009926,29.0,0.303706,0.038438,0.106319,0.035655,0.101666,766.890819,1.716394e+09,...,cc2-fix_invariant_378_zhy9uf3w,378.0,arch-cc2-1,invariant,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
3,0.802654,0.009994,8.0,0.326628,0.018057,0.044836,0.007241,0.032580,675.566714,1.716394e+09,...,cc2-fix_equivariant_378_frxln4v9,378.0,arch-cc2-1,equivariant,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
4,0.926061,0.009595,68.0,0.278383,0.011373,-0.059240,-0.056205,0.017406,156.246666,1.716394e+09,...,cc2-fix_egnn-invariant_378_28j5cxjj,378.0,arch-cc2-1,egnn-invariant,cc2-fix,test_r2 _runtime _timestamp train_...,{'runtime': 154},NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,NaN,0.000902,220.0,0.205305,NaN,NaN,NaN,NaN,7730.751498,1.716086e+09,...,invariant-yxyean2i,NaN,None,None,None,_step train_loss _runtime lr ...,NaN,NaN,NaN,0.163244
899,NaN,0.000902,220.0,0.208017,NaN,NaN,NaN,NaN,7429.813958,1.716082e+09,...,invariant-6c54xo73,NaN,None,None,None,_step train_loss _runtime lr ...,NaN,NaN,NaN,0.166301
900,NaN,0.000004,499.0,0.233126,NaN,NaN,NaN,NaN,135.476969,1.716073e+09,...,gnn,NaN,None,None,None,_timestamp train_loss lr _step...,{'runtime': 135},NaN,NaN,0.235054
901,NaN,0.000004,499.0,0.194568,NaN,NaN,NaN,NaN,154.924073,1.716074e+09,...,topological,NaN,None,None,None,_runtime eval_loss train_loss _tim...,{'runtime': 155},NaN,NaN,0.156134


## Evaluation scheme 1

Take the best val_r2 score from all the architectures for a given seed, and report the mean and std of the corresponding best_model_test_r2 and best_model_test_mse scores.

In [79]:
runs_df['lr'] = runs_df.config.apply(lambda x: x.get("optimizer", None)).apply(
    lambda x: None if x is None else x.get("lr", None)
)
runs_df

,best_model_test_mse,lr,_step,train_loss,best_val_r2,best_model_test_r2,val_r2,test_r2,_runtime,_timestamp,...,name,seed,arch,baseline,ckpt_prefix,history,_wandb,scatter,best_r2,eval_loss
0,0.809440,0.01,8.0,0.324672,0.014962,0.033230,0.003756,0.051446,718.011611,1.716394e+09,...,cc2-fix_equivariant-virtual_378_w69pf0ua,378.0,arch-cc2-1,equivariant-virtual,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
1,0.802445,0.01,8.0,0.327559,0.018571,0.045479,0.007354,0.033446,745.481871,1.716394e+09,...,cc2-fix_equivariant-ho_378_wl3jazrl,378.0,arch-cc2-1,equivariant-ho,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
2,0.766299,0.01,29.0,0.303706,0.038438,0.106319,0.035655,0.101666,766.890819,1.716394e+09,...,cc2-fix_invariant_378_zhy9uf3w,378.0,arch-cc2-1,invariant,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
3,0.802654,0.01,8.0,0.326628,0.018057,0.044836,0.007241,0.032580,675.566714,1.716394e+09,...,cc2-fix_equivariant_378_frxln4v9,378.0,arch-cc2-1,equivariant,cc2-fix,_step train_loss _runtime lr ...,NaN,NaN,NaN,NaN
4,0.926061,0.01,68.0,0.278383,0.011373,-0.059240,-0.056205,0.017406,156.246666,1.716394e+09,...,cc2-fix_egnn-invariant_378_28j5cxjj,378.0,arch-cc2-1,egnn-invariant,cc2-fix,test_r2 _runtime _timestamp train_...,{'runtime': 154},NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,NaN,NaN,220.0,0.205305,NaN,NaN,NaN,NaN,7730.751498,1.716086e+09,...,invariant-yxyean2i,NaN,None,None,None,_step train_loss _runtime lr ...,NaN,NaN,NaN,0.163244
899,NaN,NaN,220.0,0.208017,NaN,NaN,NaN,NaN,7429.813958,1.716082e+09,...,invariant-6c54xo73,NaN,None,None,None,_step train_loss _runtime lr ...,NaN,NaN,NaN,0.166301
900,NaN,NaN,499.0,0.233126,NaN,NaN,NaN,NaN,135.476969,1.716073e+09,...,gnn,NaN,None,None,None,_timestamp train_loss lr _step...,{'runtime': 135},NaN,NaN,0.235054
901,NaN,NaN,499.0,0.194568,NaN,NaN,NaN,NaN,154.924073,1.716074e+09,...,topological,NaN,None,None,None,_runtime eval_loss train_loss _tim...,{'runtime': 155},NaN,NaN,0.156134


In [195]:
import numpy as np


ckpt_filter = "cc2-fix"
# arch_filter = "arch-cc2-1"
# lr_filter = 0.001
df1 = runs_df.copy()
df1 = runs_df[
    (runs_df.ckpt_prefix == ckpt_filter) & 
    # (runs_df.arch == arch_filter) &
    # (runs_df.lr == lr_filter) &
    runs_df.val_r2.notnull() & df1.seed.notnull()
]

# print counts per seed for the equivariant-virtual and equivariant
for s in df1.seed.unique():
    for b in ["equivariant-virtual", "equivariant"]:
        tmp = df1[(df1.seed == s) & (df1.baseline == b)]
        if len(tmp) == 0:
            h_len = 0
        else:
            h_len = len(tmp.history.values[0])
        print(f"seed: {s}, baseline: {b}, len: {h_len}")

# replace equivariant-virtual and equivariant-ho with equivariant
# df1["baseline"] = df1["baseline"].replace("equivariant-virtual", "equivariant")
# df1["baseline"] = df1["baseline"].replace("equivariant-ho", "equivariant")

# df1 = df1.loc[df1.groupby(["seed", "baseline", "arch", "lr"]).val_r2.idxmax()]
df1 = df1.loc[df1.groupby(["seed", "baseline"]).val_r2.idxmax()]
df1["seed"] = df1["seed"].astype(int)

seeds = [42, 126, 294]
for s in seeds:
    print(f"Seed {s}")
    df1s = df1[df1.seed == s]
    df1s = df1s.sort_values("best_model_test_mse", ascending=False)[
        ["baseline", "best_model_test_mse"]
    ]
    display(df1s)

# average over seeds
print("Average over 3 seeds")
df1 = df1[df1.seed.isin(seeds)]
df1_avg = df1.groupby("baseline")["best_model_test_mse"].mean().reset_index()
df1_avg_std = df1.groupby("baseline")["best_model_test_mse"].std().reset_index()
df1_avg["SE"] = 1.96 * df1_avg_std["best_model_test_mse"] / np.sqrt(len(seeds))
df1_avg = df1_avg.sort_values("best_model_test_mse", ascending=False)
display(df1_avg)
# df1

seed: 378.0, baseline: equivariant-virtual, len: 9
seed: 378.0, baseline: equivariant, len: 9
seed: 168.0, baseline: equivariant-virtual, len: 16
seed: 168.0, baseline: equivariant, len: 16
seed: 84.0, baseline: equivariant-virtual, len: 0
seed: 84.0, baseline: equivariant, len: 0
seed: 0.0, baseline: equivariant-virtual, len: 103
seed: 0.0, baseline: equivariant, len: 121
seed: 336.0, baseline: equivariant-virtual, len: 85
seed: 336.0, baseline: equivariant, len: 76
seed: 126.0, baseline: equivariant-virtual, len: 87
seed: 126.0, baseline: equivariant, len: 106
seed: 42.0, baseline: equivariant-virtual, len: 163
seed: 42.0, baseline: equivariant, len: 60
seed: 294.0, baseline: equivariant-virtual, len: 90
seed: 294.0, baseline: equivariant, len: 97
Seed 42


,baseline,best_model_test_mse
317,egnn-equivariant,1.049558
318,gnn,1.018255
186,mlp,1.001868
312,linear,0.997205
319,topological-pos,0.956837
178,equivariant-virtual,0.955211
302,equivariant-pos,0.951822
247,equivariant-virtual-pos,0.939927
316,equivariant,0.925043
314,equivariant-ho,0.924918


Seed 126


,baseline,best_model_test_mse
300,linear,1.137177
155,equivariant-pos,1.006603
294,egnn-equivariant,0.911799
295,gnn,0.907583
159,equivariant,0.884385
157,equivariant-ho,0.884313
163,topological,0.883699
164,mlp,0.878199
147,invariant,0.860046
297,topological-pos,0.851902


Seed 294


,baseline,best_model_test_mse
265,linear,1.112283
233,egnn-equivariant,0.911148
226,mlp-large,0.910857
237,mlp,0.909848
272,gnn,0.908952
220,egnn-invariant,0.905379
229,equivariant-ho,0.894701
231,equivariant,0.894505
230,invariant,0.882797
266,equivariant-virtual,0.862925


Average over 3 seeds


,baseline,best_model_test_mse,SE
9,linear,1.082222,0.084499
4,equivariant-pos,0.979212,0.043833
0,egnn-equivariant,0.957501,0.090216
7,gnn,0.944930,0.071862
10,mlp,0.929972,0.072699
11,mlp-large,0.910857,NaN
1,egnn-invariant,0.905379,NaN
2,equivariant,0.901311,0.023952
3,equivariant-ho,0.901311,0.023870
8,invariant,0.888859,0.036521


Table 1

In [194]:
import numpy as np


bslns = {
    # "equivariant": "Equivariant",
    "equivariant-virtual": "ETNN",
    # "equivariant-ho": "ETNN - VN",
    # "topological": "TNN",
    "linear": "Linear",
    "gnn": "GNN",
    "egnn-equivariant": "EGNN",
    # "invariant": "ETNN - VN ",
    # "equivariant": "ETNN -- no virtual nd",
    "mlp-large": "MLP",
}

# subset the bslns above
df_tmp = df1_avg[df1_avg.baseline.isin(bslns.keys())].copy(0)
df_tmp = df_tmp.rename(columns={"best_model_test_mse": "RMSE"})
df_tmp["RMSE"] = np.sqrt(df_tmp["RMSE"].values).round(3) # .astype(str) + "%"

# remap
df_tmp["baseline"] = df_tmp["baseline"].map(bslns)

# format as latex table using pandas functionality
print(
    df_tmp.to_latex(
        # "test.tex",
        index=False,
        caption="Test",
        label="tab:test",
        column_format="ll",
        float_format="%.3f",
    )
)

\begin{table}
\caption{Test}
\label{tab:test}
\begin{tabular}{ll}
\toprule
baseline & RMSE & SE \\
\midrule
Linear & 1.040 & 0.043 \\
EGNN & 0.979 & 0.046 \\
GNN & 0.972 & 0.037 \\
MLP & 0.954 & NaN \\
ETNN & 0.939 & 0.039 \\
\bottomrule
\end{tabular}
\end{table}



In [192]:
# ablations
ablations = {
    # "equivariant": "Equivariant",
    "equivariant-virtual": "ETNN",
    "equivariant-ho": "no virtual node",
    "equivariant": "no higher-order grads.",
    "topological": "no geometric features",
    "invariant": "no position update",
}

df1_avg = df1.groupby("baseline")["best_model_test_mse"].mean().reset_index()
df1_avg_std = df1.groupby("baseline")["best_model_test_mse"].std().reset_index()
df1_avg = df1_avg.sort_values("best_model_test_mse", ascending=False)

df_tmp = df1_avg[df1_avg.baseline.isin(ablations.keys())].copy()
df_tmp["SE"] = df1_avg_std[df1_avg_std.baseline.isin(ablations.keys())].best_model_test_mse.values / np.sqrt(3)
df_tmp = df_tmp.rename(columns={"best_model_test_mse": "RMSE"})
df_tmp["RMSE"] = np.sqrt(df_tmp["RMSE"].values).round(3) # .astype(str) + "%"

df_tmp["baseline"] = df_tmp["baseline"].map(ablations)

etnn_val = df_tmp[df_tmp.baseline == "ETNN"].RMSE.values[0]
df_tmp["RMSE Loss"] = "-" + ((df_tmp.RMSE - etnn_val)/ etnn_val * 100).round(2).astype(str) + "%"

print(
    df_tmp[["baseline", "RMSE Loss", "SE"]].to_latex(
        # "ablations.tex",
        index=False,
        caption="Test",
        label="tab:ablations",
        column_format="ll",
        float_format="%.3f",
    )
)

\begin{table}
\caption{Test}
\label{tab:ablations}
\begin{tabular}{ll}
\toprule
baseline & RMSE Loss & SE \\
\midrule
no higher-order grads. & -1.06% & 0.012 \\
no virtual node & -1.06% & 0.012 \\
no position update & -0.43% & 0.039 \\
no geometric features & -0.21% & 0.019 \\
ETNN & -0.0% & 0.020 \\
\bottomrule
\end{tabular}
\end{table}

